In [1]:
import datetime
import sys
import pathlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution1D, TimeDistributed, Conv2D, MaxPooling1D, Input, concatenate
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import *
from keras.optimizers import Adam, Nadam
from keras import backend as K

sys.path.append(str(pathlib.Path.cwd().parents[0]))
from datasets.utils import calc_pivots

%matplotlib

Using TensorFlow backend.


Using matplotlib backend: Qt5Agg


## Import Tick Data and Create 5min RTH Bars

In [2]:
tick_data = pd.read_feather('../data/processed/ES_tick.feather')
tick_data = tick_data[tick_data['date'] > '2017-07-29']
#Create Index from date column
tick_data.index = tick_data['date']
tick_data.drop(labels=['date'],axis=1,inplace=True)
tick_data.tail()

,last,bid,ask,volume
date,,,,
2018-01-25 19:54:52.612000-05:00,2844.0,2843.75,2844.0,1
2018-01-25 19:54:52.615000-05:00,2844.0,2843.75,2844.0,1
2018-01-25 19:54:54.157000-05:00,2844.0,2843.75,2844.0,1
2018-01-25 19:54:54.157000-05:00,2844.0,2843.75,2844.0,1
2018-01-25 19:54:55.332000-05:00,2844.0,2843.75,2844.0,2


In [3]:
#Resample to get 5min bars
five_min_data = pd.DataFrame(
    tick_data['last'].resample('5Min', loffset=datetime.timedelta(minutes=5)).ohlc())

import pandas_market_calendars as mcal
#We hack the NYSE Calendar extending the close until 4:15
class CMERTHCalendar(mcal.exchange_calendar_nyse.NYSEExchangeCalendar):
    @property
    def close_time(self):
        return datetime.time(16, 15)
    
#Create RTH Calendar
nyse = CMERTHCalendar()
schedule = nyse.schedule(start_date=five_min_data.index.min(), 
                         end_date=five_min_data.index.max())

#Filter out those bars that occur during RTH
five_min_data['dates'] = pd.to_datetime(five_min_data.index.to_datetime().date)
five_min_data['valid_date'] = five_min_data['dates'].isin(schedule.index)
five_min_data['valid_time'] = False
during_rth = five_min_data['valid_date'] & \
        (five_min_data.index > schedule.loc[five_min_data['dates'],'market_open']) & \
        (five_min_data.index <= schedule.loc[five_min_data['dates'],'market_close'])
five_min_data.loc[during_rth, 'valid_time'] = True
five_min_data = five_min_data[five_min_data['valid_time'] == True]
five_min_data.drop(['dates','valid_date','valid_time'], axis=1, inplace=True)

#Add ema
five_min_data['ema'] = five_min_data['close'].ewm(span=20, min_periods=20).mean()

#Reset index
five_min_data.reset_index(inplace=True)

five_min_data[81:].head()

,date,open,high,low,close,ema
81,2017-08-01 09:35:00-04:00,2475.50,2476.00,2472.50,2473.50,2470.908594
82,2017-08-01 09:40:00-04:00,2473.50,2474.00,2471.50,2472.50,2471.060194
83,2017-08-01 09:45:00-04:00,2472.25,2473.25,2471.75,2473.00,2471.244978
84,2017-08-01 09:50:00-04:00,2473.00,2473.25,2472.00,2472.75,2471.388343
85,2017-08-01 09:55:00-04:00,2472.75,2473.00,2471.25,2471.25,2471.375165


In [83]:
#Add column for number of seconds elapsed in trading day
five_min_data['sec'] = (five_min_data['date'].values 
                        - five_min_data['date'].values.astype('datetime64[D]')) / np.timedelta64(1,'s')

#Calculate sin & cos time
#24hr time is a cyclical continuous feature
seconds_in_day = 24*60*60
five_min_data['sin_time'] = np.sin(2*np.pi*five_min_data['sec']/seconds_in_day)
five_min_data['cos_time'] = np.cos(2*np.pi*five_min_data['sec']/seconds_in_day)

five_min_data.drop('sec', axis=1, inplace=True)
five_min_data.head()

,date,open,high,low,close,ema,sin_time,cos_time
0,2017-07-31 09:35:00-04:00,2474.75,2475.75,2474.00,2475.50,NaN,-0.402747,-0.915311
1,2017-07-31 09:40:00-04:00,2475.25,2476.00,2473.75,2475.50,NaN,-0.422618,-0.906308
2,2017-07-31 09:45:00-04:00,2475.75,2475.75,2474.50,2474.75,NaN,-0.442289,-0.896873
3,2017-07-31 09:50:00-04:00,2474.50,2475.00,2473.50,2473.75,NaN,-0.461749,-0.887011
4,2017-07-31 09:55:00-04:00,2474.00,2474.25,2472.75,2472.75,NaN,-0.480989,-0.876727


## Create Test / Train Datasets

In [107]:
five_min_data = pd.read_feather('../data/processed/ES_TFCnn.feather')
five_min_data.head()

,date,open,high,low,close,ema,sin_time,cos_time,btc,stc
0,2017-07-31 09:35:00-04:00,2474.75,2475.75,2474.00,2475.50,NaN,-0.402747,-0.915311,-104,96
1,2017-07-31 09:40:00-04:00,2475.25,2476.00,2473.75,2475.50,NaN,-0.422618,-0.906308,-104,96
2,2017-07-31 09:45:00-04:00,2475.75,2475.75,2474.50,2474.75,NaN,-0.442289,-0.896873,-104,96
3,2017-07-31 09:50:00-04:00,2474.50,2475.00,2473.50,2473.75,NaN,-0.461749,-0.887011,-104,96
4,2017-07-31 09:55:00-04:00,2474.00,2474.25,2472.75,2472.75,NaN,-0.480989,-0.876727,-104,96


In [3]:
five_min_data = pd.read_hdf('../data/processed/store.h5', key='cnn_data')
#five_min_data = calc_pivots(five_min_data)
five_min_data.head()

,date,open,high,low,close,ema,sin_time,cos_time,btc,stc
0,2006-01-03 09:35:00-05:00,1259.00,1260.75,1259.00,1259.75,NaN,-0.625923,-0.779884,-104,96
1,2006-01-03 09:40:00-05:00,1259.50,1259.75,1258.50,1259.50,NaN,-0.642788,-0.766044,-104,96
2,2006-01-03 09:45:00-05:00,1259.25,1260.25,1259.25,1260.00,NaN,-0.659346,-0.751840,-104,96
3,2006-01-03 09:50:00-05:00,1259.75,1260.00,1258.50,1258.75,NaN,-0.675590,-0.737277,-104,96
4,2006-01-03 09:55:00-05:00,1259.00,1259.75,1257.50,1257.75,NaN,-0.691513,-0.722364,96,-104


In [8]:
five_min_data.dtypes

date          datetime64[ns, America/New_York]
open                                   float32
high                                   float32
low                                    float32
close                                  float32
ema                                    float64
sin_time                               float64
cos_time                               float64
btc                                      int64
stc                                      int64
ph                                     float64
pl                                     float64
swing                                  float64
change                                 float32
cdl_sign                               float32
cdl_body                               float32
cdl_ut                                 float32
cdl_lt                                 float32
cdl_rng                                float32
cdl_hl                                   int64
cdl_lh                                   int64
pivot_high   

In [83]:
WINDOW = 36
EMB_SIZE = 7

i_start = 19



for i_start in range(19, five_min_data.shape[0]-WINDOW):
    i_end = i_start + WINDOW + 1
    wd = five_min_data[i_start:i_end].copy()

    last_bar_close = wd.iloc[-1]['close']
    last_bar_high = wd.iloc[-1]['high']
    last_bar_open = wd.iloc[-1]['open']

    wd.loc[:, ['open','high','low','close','ema','ph','pl']] = wd.loc[:, ['open','high','low',
                                                                          'close','ema','ph','pl']] / last_bar_close
    x_i = wd.loc[:, ['open','high','low','close','ema','sin_time','cos_time']].values.tolist()
    x2_i = []
    for i, r in wd.iterrows():
        if not pd.isnull(r['ph']):
            x2_i.append([r['sin_time'], r['cos_time'], r['ph'], 1])
        if not pd.isnull(r['pl']):
            x2_i.append([r['sin_time'], r['cos_time'], r['pl'], 0])
        
    if wd.iloc[-1]['btc'] > 0:
        y_i = [1, 0]
    else:
        y_i = [0, 1]


wd.head()
for i_start in range(19, five_min_data.shape[0]-WINDOW):
    print(i)

19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


In [11]:
fd = five_min_data
fd['ema_5'] = fd['close'].ewm(span=5, min_periods=5).mean()
fd['ema_3'] = fd['close'].ewm(span=3, min_periods=3).mean()

fd['change'] = fd['close'] - fd['close'].shift(1)
fd['cdl_hl'] = np.where(fd['low'] >= fd['low'].shift(), 1, 0) #higher low
fd['cdl_lh'] = np.where(fd['high'] <= fd['high'].shift(), 1, 0) #lower high

fd.head()

,date,open,high,low,close,ema,sin_time,cos_time,btc,stc,ema_5,ema_3,change,cdl_hl,cdl_lh
0,2006-01-03 09:35:00-05:00,1259.00,1260.75,1259.00,1259.75,NaN,-0.625923,-0.779884,-104,96,NaN,NaN,NaN,0,0
1,2006-01-03 09:40:00-05:00,1259.50,1259.75,1258.50,1259.50,NaN,-0.642788,-0.766044,-104,96,NaN,NaN,-0.25,0,1
2,2006-01-03 09:45:00-05:00,1259.25,1260.25,1259.25,1260.00,NaN,-0.659346,-0.751840,-104,96,NaN,1259.821429,0.50,1,0
3,2006-01-03 09:50:00-05:00,1259.75,1260.00,1258.50,1258.75,NaN,-0.675590,-0.737277,-104,96,NaN,1259.250000,-1.25,0,1
4,2006-01-03 09:55:00-05:00,1259.00,1259.75,1257.50,1257.75,NaN,-0.691513,-0.722364,96,-104,1258.740521,1258.475806,-1.00,0,1


In [24]:
#fd['cdl_sign'] = np.sign(fd['close'] - fd['open'])
fd['cdl_body'] = (fd['close'] - fd['open']) / fd['close'] * 100
fd['cdl_rng'] = fd['high'] - fd['low']
fd['cdl_ut'] = np.where(fd['cdl_body'] > 0, fd['high'] - fd['close'], fd['high'] - fd['open'])
fd['cdl_lt'] = np.where(fd['cdl_body'] > 0, fd['open'] - fd['low'], fd['close'] - fd['low'])
fd['cdl_ut'] = np.where(fd['cdl_rng'] == 0, 0, fd['cdl_ut'] / fd['cdl_rng'])
fd['cdl_lt'] = np.where(fd['cdl_rng'] == 0, 0, fd['cdl_lt'] / fd['cdl_rng'])
fd['cdl_rng'] = (fd['cdl_rng'] / fd['low']) * 100
fd.head()

,date,open,high,low,close,ema,sin_time,cos_time,btc,stc,ema_5,ema_3,change,cdl_hl,cdl_lh,cdl_body,cdl_rng,cdl_ut,cdl_lt
0,2006-01-03 09:35:00-05:00,1259.00,1260.75,1259.00,1259.75,NaN,-0.625923,-0.779884,-104,96,NaN,NaN,NaN,0,0,0.059536,0.138999,0.571429,0.000000
1,2006-01-03 09:40:00-05:00,1259.50,1259.75,1258.50,1259.50,NaN,-0.642788,-0.766044,-104,96,NaN,NaN,-0.25,0,1,0.000000,0.099325,0.200000,0.800000
2,2006-01-03 09:45:00-05:00,1259.25,1260.25,1259.25,1260.00,NaN,-0.659346,-0.751840,-104,96,NaN,1259.821429,0.50,1,0,0.059524,0.079412,0.250000,0.000000
3,2006-01-03 09:50:00-05:00,1259.75,1260.00,1258.50,1258.75,NaN,-0.675590,-0.737277,-104,96,NaN,1259.250000,-1.25,0,1,-0.079444,0.119190,0.166667,0.166667
4,2006-01-03 09:55:00-05:00,1259.00,1259.75,1257.50,1257.75,NaN,-0.691513,-0.722364,96,-104,1258.740521,1258.475806,-1.00,0,1,-0.099384,0.178926,0.333333,0.111111


In [96]:
higher_than_next_bar = fd['high'] > fd.shift(-1)['high']
higher_than_prev_bar = fd['high'] > fd.shift(1)['high']
lower_than_next_bar = fd['low'] < fd.shift(-1)['low']
lower_than_prev_bar = fd['low'] < fd.shift(1)['low']
fd['pivot_high'] = higher_than_next_bar & higher_than_prev_bar
fd['pivot_low'] = lower_than_next_bar & lower_than_prev_bar
fd.head()

,date,open,high,low,close,ema,sin_time,cos_time,btc,stc,...,change,cdl_sign,cdl_body,cdl_ut,cdl_lt,cdl_rng,cdl_hl,cdl_lh,pivot_high,pivot_low
0,2006-01-03 09:35:00-05:00,1259.00,1260.75,1259.00,1259.75,NaN,-0.625923,-0.779884,-104,96,...,NaN,1.0,0.75,1.00,0.00,1.75,0,0,False,False
1,2006-01-03 09:40:00-05:00,1259.50,1259.75,1258.50,1259.50,NaN,-0.642788,-0.766044,-104,96,...,-0.25,0.0,0.00,0.25,1.00,1.25,0,1,False,True
2,2006-01-03 09:45:00-05:00,1259.25,1260.25,1259.25,1260.00,NaN,-0.659346,-0.751840,-104,96,...,0.50,1.0,0.75,0.25,0.00,1.00,1,0,True,False
3,2006-01-03 09:50:00-05:00,1259.75,1260.00,1258.50,1258.75,NaN,-0.675590,-0.737277,-104,96,...,-1.25,-1.0,1.00,0.25,0.25,1.50,0,1,False,False
4,2006-01-03 09:55:00-05:00,1259.00,1259.75,1257.50,1257.75,NaN,-0.691513,-0.722364,96,-104,...,-1.00,-1.0,1.25,0.75,0.25,2.25,0,1,False,False


In [97]:
data = fd[81:]

openp = data['open'].tolist()
highp = data['high'].tolist()
lowp = data['low'].tolist()
closep = data['close'].tolist()
emap = data['ema'].tolist()
sin_time = data['sin_time'].tolist()
cos_time = data['cos_time'].tolist()
btc = data['btc'].tolist()
stc = data['stc'].tolist()

change = data['change'].tolist()

cdl_sign = data['cdl_sign'].tolist()
cdl_body = data['cdl_body'].tolist()
cdl_ut = data['cdl_ut'].tolist()
cdl_lt = data['cdl_lt'].tolist()
cdl_rng = data['cdl_rng'].tolist()
cdl_hl = data['cdl_hl'].tolist()
cdl_lh = data['cdl_lh'].tolist()

pivot_high = data['pivot_high'].astype('int').tolist()
pivot_low = data['pivot_low'].astype('int').tolist()

In [58]:
closep = (data['close'].shift(-1) - data['close']).tolist()[:-1]
btc = data['btc'].shift(-1).tolist()[:-1]
sin_time = data['sin_time'].shift(-1).tolist()[:-1]
cos_time = data['cos_time'].shift(-1).tolist()[:-1]

In [173]:
p = int(data.shape[0] * 0.9)
p = 10000
mean = data.mean(axis=0)
std = data.std(axis=0)
mean_c = np.mean(closep[0:p])
std_c = np.std(closep[0:p])
mean_c, std_c


(1288.2096, 21.484784856730588)

In [101]:
WINDOW = 36 #Number of bars in a trading day
EMB_SIZE = 6
STEP = 1
FORECAST = 1

X, Y = [], []
for i in range(0, len(data)-WINDOW+1, STEP):
    try:
        o = openp[i:i+WINDOW]
        h = highp[i:i+WINDOW]
        l = lowp[i:i+WINDOW]
        c = closep[i:i+WINDOW]
        e = emap[i:i+WINDOW]
        ct = cos_time[i:i+WINDOW]
        st = sin_time[i:i+WINDOW]
        
        cng = change[i:i+WINDOW]
        
        _cdl_sign = cdl_sign[i:i+WINDOW]
        _cdl_body = cdl_body[i:i+WINDOW]
        _cdl_ut = cdl_ut[i:i+WINDOW]
        _cdl_lt = cdl_lt[i:i+WINDOW]
        _cdl_rng = cdl_rng[i:i+WINDOW]
        _cdl_hl = cdl_hl[i:i+WINDOW]
        _cdl_lh = cdl_lh[i:i+WINDOW]
        
        _pivot_high = np.array(pivot_high[i:i+WINDOW]) * np.array(h)
        _pivot_low = np.array(pivot_low[i:i+WINDOW]) * np.array(l)
        
        o = (np.array(o) - np.mean(o)) / np.std(o)
        h = (np.array(h) - np.mean(h)) / np.std(h)
        l = (np.array(l) - np.mean(l)) / np.std(l)
        c = (np.array(c) - np.mean(c)) / np.std(c)
        e = (np.array(e) - np.mean(e)) / np.std(e)
        
        ph = (np.array(_pivot_high) - np.mean(h)) / np.std(h)
        pl = (np.array(_pivot_low) - np.mean(l)) / np.std(l)
        
        _cng = (np.array(cng) - np.mean(cng)) / np.std(cng)
        
        
        #c = (np.array(c) - mean_c) / std_c
        
        #o = np.divide(o, c[-1]) 
        #h = np.divide(h, c[-1])
        #l = np.divide(l, c[-1])
        #e = np.divide(e, c[-1])
        #c = np.divide(c, c[-1])

        x_i = closep[i:i+WINDOW]
        y_i = closep[(i+WINDOW-1)+FORECAST]  

        last_close = x_i[-1]
        next_close = y_i

        if btc[i+WINDOW-1] > 0:
            y_i = [1, 0]
        else:
            y_i = [0, 1]
        
        args = (c, )
        args += (o, h, l)
        #x_i = np.column_stack((o, h, l, c, e, ct, st, cng, _cdl_hl))
        #x_i = np.column_stack((cng))
        x_i = np.column_stack((c, e, ct, st, _cdl_lh, _cdl_hl))
        
    except Exception as e:
        #e.throw()
        break

    #only add if 1pt body and close on high
    if (closep[i+WINDOW-1] == highp[i+WINDOW-1]) and (closep[i+WINDOW-1]-openp[i+WINDOW-1]>=1):
        X.append(x_i)
        Y.append(y_i)

In [18]:
X

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [116]:
# Let's split into train and test sets
# Train Set will be from 8/1/17 through 12/31/17, Test Set 1/1/17 - 1/25/17
p = 8547 #Manual split for now
p = int(len(X) * 0.9)
X, Y = np.array(X), np.array(Y)
X_train = X[0:p]
Y_train = Y[0:p]
X_test = X[p:]
Y_test = Y[p:]

#We may want to shuffle the training data -- will look into this later
def shuffle_in_unison(a, b):
    # courtsey http://stackoverflow.com/users/190280/josh-bleecher-snyder
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

#X_train, Y_train = shuffle_in_unison(X_train, Y_train)

# Not sure why this is needed, but we apply it anyway
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1], EMB_SIZE))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1], EMB_SIZE))
X_train[-1]

array([[[ 3.86654447e-01,  9.89805943e-01, -5.73576436e-01,
         -8.19152044e-01,  1.00000000e+00,  0.00000000e+00],
        [-7.73308893e-02,  9.04456241e-01, -5.55570233e-01,
         -8.31469612e-01,  1.00000000e+00,  0.00000000e+00],
        [ 7.73308893e-02,  8.63709197e-01, -5.37299608e-01,
         -8.43391446e-01,  1.00000000e+00,  1.00000000e+00],
        [ 2.31992668e-01,  8.63316938e-01, -5.18773258e-01,
         -8.54911871e-01,  0.00000000e+00,  1.00000000e+00],
        [-5.41316225e-01,  6.80591464e-01, -5.00000000e-01,
         -8.66025404e-01,  1.00000000e+00,  0.00000000e+00],
        [-8.50639783e-01,  4.42320187e-01, -4.80988769e-01,
         -8.76726756e-01,  1.00000000e+00,  0.00000000e+00],
        [-2.31992668e-01,  3.72637871e-01, -4.61748613e-01,
         -8.87010833e-01,  0.00000000e+00,  1.00000000e+00],
        [-8.50639783e-01,  1.63695508e-01, -4.42288690e-01,
         -8.96872742e-01,  1.00000000e+00,  1.00000000e+00],
        [-2.31992668e-01,  1.205

## Train CNN Model

In [52]:
model = Sequential()

model.add(
    TimeDistributed(
        Conv2D(32, (7, 7), padding='same', strides=2),
        input_shape=(None, 540, 960, 2)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_53 (TimeDis (None, None, 270, 480, 32 4736      
Total params: 4,736
Trainable params: 4,736
Non-trainable params: 0
_________________________________________________________________


In [147]:
model = Sequential()
model.add(TimeDistributed(Convolution1D(filters=32,
                                        kernel_size=6,
                                        padding='same',
                                        activation='relu'),
                          input_shape = (None, WINDOW, EMB_SIZE)))
#model.add(TimeDistributed(BatchNormalization()))
#model.add(TimeDistributed(LeakyReLU()))
model.add(TimeDistributed(Dropout(0.75)))

model.add(TimeDistributed(Convolution1D(filters=64,
                        kernel_size=12,
                        padding='same',
                        activation='relu')))
#model.add(TimeDistributed(BatchNormalization()))
#model.add(TimeDistributed(LeakyReLU()))
model.add(TimeDistributed(Dropout(0.75)))

model.add(TimeDistributed(Flatten()))

model.add(TimeDistributed((Dense(32, activation='relu'))))
#model.add(TimeDistributed(BatchNormalization()))
#model.add(TimeDistributed(LeakyReLU()))

model.add(LSTM(64, recurrent_dropout=0.75, stateful=False))
#model.add(Dropout(0))


model.add(Dense(2, activation='softmax'))
#model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_79 (TimeDis (None, None, 36, 32)      1184      
_________________________________________________________________
time_distributed_80 (TimeDis (None, None, 36, 32)      0         
_________________________________________________________________
time_distributed_81 (TimeDis (None, None, 36, 64)      24640     
_________________________________________________________________
time_distributed_82 (TimeDis (None, None, 36, 64)      0         
_________________________________________________________________
time_distributed_83 (TimeDis (None, None, 2304)        0         
_________________________________________________________________
time_distributed_84 (TimeDis (None, None, 32)          73760     
_________________________________________________________________
lstm_18 (LSTM)               (None, 64)                24832     
__________

In [2]:
from datasets.es_btc import ESBTCDataset
ds = ESBTCDataset(use_cdl=True, use_ema=False, use_ema5=True)
ds.load_or_generate_data()

In [6]:
model = Sequential()
model.add(Convolution1D(input_shape = ds.input_shape,
                        filters=32,
                        kernel_size=6,
                        padding='same',
                        activation='relu'))
#model.add(BatchNormalization())
#model.add(LeakyReLU())
#model.add(MaxPooling1D(strides=1))
model.add(Dropout(0.75))

model.add(Convolution1D(filters=64,
                        kernel_size=12,
                        padding='same',
                        activation='relu'))
#model.add(BatchNormalization())
#model.add(LeakyReLU())
#model.add(MaxPooling1D(strides=1))
model.add(Dropout(0.75))

model.add(Flatten())

model.add(Dense(32, activation='relu'))
#model.add(BatchNormalization())
#model.add(LeakyReLU())
#model.add(Dropout(0.75))

model.add(Dense(2, activation='softmax'))
#model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 36, 32)            1568      
_________________________________________________________________
dropout_3 (Dropout)          (None, 36, 32)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 36, 64)            24640     
_________________________________________________________________
dropout_4 (Dropout)          (None, 36, 64)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                73760     
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 66        
Total para

In [82]:
model = Sequential()
model.add(LSTM(64, return_sequences=False,
               input_shape=ds.input_shape,
              dropout=0,
              recurrent_dropout=0.75))  # returns a sequence of vectors of dimension 32
#model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
#model.add(LSTM(32, recurrent_dropout=0.75))  # return a single vector of dimension 32
model.add(Dense(2, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                18176     
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 130       
Total params: 18,306
Trainable params: 18,306
Non-trainable params: 0
_________________________________________________________________


In [90]:
cnn_input = Input(shape=ds.input_shape, dtype='float32', name='cnn_input')
x = Convolution1D(filters=32,
                  kernel_size=6,
                  padding='same',
                  activation='relu')(cnn_input)
x = Dropout(0.75)(x)
x = Convolution1D(filters=64,
                  kernel_size=12,
                  padding='same',
                  activation='relu')(x)
x = Dropout(0.75)(x)
x = Flatten()(x)
x = Dense(32, activation='relu')(x)

x1 = LSTM(32, recurrent_dropout=0.75)(cnn_input)

merged = concatenate([x, x1], axis=-1)
x2 = Dense(32, activation='relu')(merged)

output = Dense(2, activation='softmax', name='output')(x2)
model = Model(inputs=[cnn_input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cnn_input (InputLayer)          (None, 36, 6)        0                                            
__________________________________________________________________________________________________
conv1d_25 (Conv1D)              (None, 36, 32)       1184        cnn_input[0][0]                  
__________________________________________________________________________________________________
dropout_25 (Dropout)            (None, 36, 32)       0           conv1d_25[0][0]                  
__________________________________________________________________________________________________
conv1d_26 (Conv1D)              (None, 36, 64)       24640       dropout_25[0][0]                 
__________________________________________________________________________________________________
dropout_26

In [7]:
ds.x_train.shape

(0, 36)

In [7]:
def precision_threshold(threshold=0.5):
    def precision(y_true, y_pred):
        """Precision metric.
        Computes the precision over the whole batch using threshold_value.
        """
        
        y_true = y_true[:, 0]
        y_pred = y_pred[:, 0]
        
        threshold_value = threshold
        # Adaptation of the "round()" used before to get the predictions. Clipping to make sure that the predicted raw values are between 0 and 1.
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        # Compute the number of true positives. Rounding in prevention to make sure we have an integer.
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        # count the predicted positives
        predicted_positives = K.sum(y_pred)
        # Get the precision ratio
        precision_ratio = true_positives / (predicted_positives + K.epsilon())
        return precision_ratio
    return precision

def precision(y_true, y_pred):
    '''Calculates the precision, a metric for multi-label classification of
    how many selected items are relevant.
    '''
    
    y_true = y_true[:, 0]
    y_pred = y_pred[:, 0]
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

class Metrics(Callback):

    def on_train_begin(self, logs={}):
        self.val_precisions = []
        self.val_profits = []
 
    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()[:,0]
        val_targ = self.validation_data[1][:,0]
        
        true_positives = np.sum(np.round(val_predict * val_targ))
        false_positives = np.sum(np.round(val_predict * (1-val_targ)))
        predicted_positives = np.sum(np.round(val_predict))
        _val_precision = true_positives / (predicted_positives + 1e-7)
        _val_profit = 96 * true_positives - 104 * false_positives
        self.val_precisions.append(_val_precision)
        self.val_profits.append(_val_profit)
        print (f'- val_precision: {_val_precision} - val_profit: {_val_profit}')
        return
        
metrics = Metrics()

opt = Adam(lr=1e-4)

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.9, patience=30, min_lr=0.000001, verbose=1)
checkpointer = ModelCheckpoint(filepath="model.hdf5", verbose=1, save_best_only=True)


model.compile(optimizer=opt, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(ds.x_train, ds.y_train, 
          epochs = 250, 
          batch_size = 16384, 
          verbose=1, 
          validation_data=(ds.x_test, ds.y_test),
          callbacks=[metrics, checkpointer],
          shuffle=True)

Train on 227888 samples, validate on 25321 samples
Epoch 1/250
227888/227888 [==============================] - 4s 17us/step - loss: 0.7382 - acc: 0.5009 - val_loss: 0.6931 - val_acc: 0.5074
- val_precision: 0.49994123868257206 - val_profit: -34136.0

Epoch 00001: val_loss improved from inf to 0.69312, saving model to model.hdf5
Epoch 2/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.7114 - acc: 0.5003 - val_loss: 0.6929 - val_acc: 0.5099
- val_precision: 0.5045583619712366 - val_profit: -20664.0

Epoch 00002: val_loss improved from 0.69312 to 0.69289, saving model to model.hdf5
Epoch 3/250
227888/227888 [==============================] - 2s 11us/step - loss: 0.7022 - acc: 0.5021 - val_loss: 0.6930 - val_acc: 0.5122
- val_precision: 0.5074386312855512 - val_profit: -20264.0

Epoch 00003: val_loss did not improve from 0.69289
Epoch 4/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6982 - acc: 0.5041 - val_loss: 0.6930 - val_acc: 0.

227888/227888 [==============================] - 3s 11us/step - loss: 0.6929 - acc: 0.5121 - val_loss: 0.6930 - val_acc: 0.5075
- val_precision: 0.0 - val_profit: 0.0

Epoch 00035: val_loss did not improve from 0.69289
Epoch 36/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6929 - acc: 0.5119 - val_loss: 0.6930 - val_acc: 0.5075
- val_precision: 0.0 - val_profit: 0.0

Epoch 00036: val_loss did not improve from 0.69289
Epoch 37/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6929 - acc: 0.5122 - val_loss: 0.6930 - val_acc: 0.5075
- val_precision: 0.0 - val_profit: 0.0

Epoch 00037: val_loss did not improve from 0.69289
Epoch 38/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6929 - acc: 0.5124 - val_loss: 0.6930 - val_acc: 0.5075
- val_precision: 0.0 - val_profit: 0.0

Epoch 00038: val_loss did not improve from 0.69289
Epoch 39/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.

227888/227888 [==============================] - 3s 11us/step - loss: 0.6928 - acc: 0.5128 - val_loss: 0.6930 - val_acc: 0.5075
- val_precision: 0.0 - val_profit: 0.0

Epoch 00070: val_loss did not improve from 0.69289
Epoch 71/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6928 - acc: 0.5129 - val_loss: 0.6930 - val_acc: 0.5075
- val_precision: 0.0 - val_profit: 0.0

Epoch 00071: val_loss did not improve from 0.69289
Epoch 72/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6928 - acc: 0.5129 - val_loss: 0.6930 - val_acc: 0.5075
- val_precision: 0.0 - val_profit: 0.0

Epoch 00072: val_loss did not improve from 0.69289
Epoch 73/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6928 - acc: 0.5131 - val_loss: 0.6930 - val_acc: 0.5075
- val_precision: 0.0 - val_profit: 0.0

Epoch 00073: val_loss did not improve from 0.69289
Epoch 74/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.

227888/227888 [==============================] - 3s 11us/step - loss: 0.6928 - acc: 0.5130 - val_loss: 0.6929 - val_acc: 0.5075
- val_precision: 0.0 - val_profit: 0.0

Epoch 00105: val_loss did not improve from 0.69289
Epoch 106/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6928 - acc: 0.5131 - val_loss: 0.6929 - val_acc: 0.5075
- val_precision: 0.0 - val_profit: 0.0

Epoch 00106: val_loss did not improve from 0.69289
Epoch 107/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6927 - acc: 0.5134 - val_loss: 0.6929 - val_acc: 0.5075
- val_precision: 0.0 - val_profit: 0.0

Epoch 00107: val_loss did not improve from 0.69289
Epoch 108/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6927 - acc: 0.5130 - val_loss: 0.6929 - val_acc: 0.5075
- val_precision: 0.0 - val_profit: 0.0

Epoch 00108: val_loss did not improve from 0.69289
Epoch 109/250
227888/227888 [==============================] - 3s 11us/step - loss

227888/227888 [==============================] - 3s 11us/step - loss: 0.6927 - acc: 0.5133 - val_loss: 0.6929 - val_acc: 0.5075
- val_precision: 0.0 - val_profit: 0.0

Epoch 00140: val_loss did not improve from 0.69289
Epoch 141/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6927 - acc: 0.5133 - val_loss: 0.6929 - val_acc: 0.5075
- val_precision: 0.0 - val_profit: 0.0

Epoch 00141: val_loss did not improve from 0.69289
Epoch 142/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6927 - acc: 0.5131 - val_loss: 0.6929 - val_acc: 0.5075
- val_precision: 0.0 - val_profit: 0.0

Epoch 00142: val_loss did not improve from 0.69289
Epoch 143/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6927 - acc: 0.5134 - val_loss: 0.6929 - val_acc: 0.5075
- val_precision: 0.0 - val_profit: 0.0

Epoch 00143: val_loss did not improve from 0.69289
Epoch 144/250
227888/227888 [==============================] - 3s 11us/step - loss

227888/227888 [==============================] - 3s 11us/step - loss: 0.6926 - acc: 0.5135 - val_loss: 0.6929 - val_acc: 0.5075
- val_precision: 0.4999999916666668 - val_profit: -24.0

Epoch 00175: val_loss did not improve from 0.69289
Epoch 176/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6926 - acc: 0.5131 - val_loss: 0.6929 - val_acc: 0.5075
- val_precision: 0.4999999750000013 - val_profit: -8.0

Epoch 00176: val_loss improved from 0.69289 to 0.69288, saving model to model.hdf5
Epoch 177/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6926 - acc: 0.5131 - val_loss: 0.6929 - val_acc: 0.5068
- val_precision: 0.3448275856123662 - val_profit: -2032.0

Epoch 00177: val_loss did not improve from 0.69288
Epoch 178/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6927 - acc: 0.5127 - val_loss: 0.6929 - val_acc: 0.5066
- val_precision: 0.4335260113101006 - val_profit: -2992.0

Epoch 00178: val_loss did not 

227888/227888 [==============================] - 3s 11us/step - loss: 0.6925 - acc: 0.5139 - val_loss: 0.6929 - val_acc: 0.5093
- val_precision: 0.5060928182394064 - val_profit: -10728.0

Epoch 00207: val_loss did not improve from 0.69288
Epoch 208/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6926 - acc: 0.5130 - val_loss: 0.6929 - val_acc: 0.5106
- val_precision: 0.507738095228021 - val_profit: -12360.0

Epoch 00208: val_loss did not improve from 0.69288
Epoch 209/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6926 - acc: 0.5127 - val_loss: 0.6929 - val_acc: 0.5101
- val_precision: 0.5069817400535557 - val_profit: -12120.0

Epoch 00209: val_loss did not improve from 0.69288
Epoch 210/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6925 - acc: 0.5145 - val_loss: 0.6929 - val_acc: 0.5099
- val_precision: 0.5061425061321428 - val_profit: -13536.0

Epoch 00210: val_loss did not improve from 0.69288
Epo

- val_precision: 0.4968963060903948 - val_profit: -45408.0

Epoch 00239: val_loss did not improve from 0.69288
Epoch 240/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6924 - acc: 0.5144 - val_loss: 0.6930 - val_acc: 0.5060
- val_precision: 0.4978350045522124 - val_profit: -38904.0

Epoch 00240: val_loss did not improve from 0.69288
Epoch 241/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6924 - acc: 0.5156 - val_loss: 0.6930 - val_acc: 0.5060
- val_precision: 0.4978603603547538 - val_profit: -39320.0

Epoch 00241: val_loss did not improve from 0.69288
Epoch 242/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6924 - acc: 0.5150 - val_loss: 0.6930 - val_acc: 0.5064
- val_precision: 0.4984706139338161 - val_profit: -39416.0

Epoch 00242: val_loss did not improve from 0.69288
Epoch 243/250
227888/227888 [==============================] - 3s 11us/step - loss: 0.6924 - acc: 0.5152 - val_loss: 0.6930 - val_

In [8]:
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

plt.figure()
plt.plot(history.history['precision'])
plt.plot(history.history['val_precision'])
plt.title('precision')
plt.ylabel('precision')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

KeyError: 'precision'

In [58]:
from sklearn.metrics import confusion_matrix

#model.load_weights("model.hdf5")
pred = model.predict(np.array(ds.x_test), batch_size=64)

C = confusion_matrix([np.argmax(y) for y in ds.y_test], [np.argmax(y) for y in pred])

print (C / C.astype(np.float).sum(axis=1)[:, None])

[[0. 1.]
 [0. 1.]]


In [59]:
C

array([[  0, 484],
       [  0, 580]])

In [27]:
#K.clip(pred[0] * Y_test[0], 0, 1)
8 / (K.epsilon()  + 14)

0.5714285673469388

In [14]:
#sum([np.argmax(y) for y in Y_test])
pd.DataFrame(pred)[0].plot()

In [28]:
data.iloc[43650,:]
pred.shape

(227844, 2)

In [38]:
pred = model.predict(np.array(ds.x_test), batch_size=64)
df = pd.DataFrame(np.concatenate((pred, ds.y_test), axis=1))
#df[(df[0]>.6)&(df[2]==1)].shape
#df[(df[0]>.6)].shape

#df.shape
df.sort_values(0,axis=0,ascending=False).head(20)
pred[:,0], 0.5

<tf.Tensor 'Greater:0' shape=(1064,) dtype=bool>

In [51]:
pred0 = pred[:,0]
y0 = ds.y_test[:, 0]
true_positives = np.sum(np.round(pred0 * y0))
false_positives = np.sum(np.round(pred0 * (1-y0)))
predicted_positives = np.sum(np.round(pred0))
np.sum(false_positives)

17.0

In [45]:
C / C.astype(np.float).sum(axis=1)[:, None]

array([[0.83673469, 0.15855573, 0.00470958],
       [0.78915663, 0.20481928, 0.0060241 ],
       [0.86335404, 0.13043478, 0.00621118]])

In [69]:
probs = Y_train.sum(axis=0) / Y_train.shape[0]
probs

array([0.55539956, 0.44460044])

In [68]:
pred

array([[0.4436597 , 0.5563404 ],
       [0.5040323 , 0.49596766],
       [0.5359964 , 0.4640036 ],
       ...,
       [0.57311577, 0.4268842 ],
       [0.5362818 , 0.46371824],
       [0.5139088 , 0.48609126]], dtype=float32)

In [120]:
s = np.random.binomial(1, probs[1], pred.shape[0])
s

array([1, 0, 1, ..., 0, 0, 1])

In [121]:
C1 = confusion_matrix([np.argmax(y) for y in Y_test], s)
print (C1 / C1.astype(np.float).sum(axis=1)[:, None])

[[0.58421851 0.41578149]
 [0.59026688 0.40973312]]


In [122]:
([np.argmax(y) for y in Y_test] == s).sum() / pred.shape[0]

0.4984567901234568